### Introduction

A URL can include either **Hypertext Transfer Protocol (HTTP)** or the **Hypertext Transfer Protocol Secure (HTTPS)**. Other types of protocols include the **File Transfer Protocol (FTP)**, **Siimple Mail Transfer (SMTP)**, and others, such as telnet, DNS, and so on.

A URL consists of the top-level domain, hostname, paths, and port of the web address.

A Lousy URLs are URLs that have been created with malicious intent. They are often the precursors to cyberattacjs that may happen in the near future. 

In [20]:
import pandas as pd
import numpy as np
import random 
import pickle

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

import re
from urllib.parse import urlparse
import urllib
from xml.dom import minidom 
import csv 
import pygeoip


In [ ]:
def url_cleanse(web_url):
    web_url = web_url.lower()  # Convert to lowercase
    parsed_url = urlparse(web_url)
    
    # Extract domain and path separately
    domain_parts = re.split(r'[.\-]', parsed_url.netloc)  # Split by dot & hyphen
    path_parts = re.split(r'[\/\-]', parsed_url.path)  # Split by slash & hyphen
    
    # Remove common TLDs and empty strings
    common_tlds = {'com', 'net', 'org', 'gov', 'edu', 'co', 'uk', 'us'}
    url_tokens = [part for part in (domain_parts + path_parts) if part and part not in common_tlds]

    return list(set(url_tokens))  # Return unique tokens

In [ ]:
test = url_cleanse("https://www.Iamstrongerthanthe_mountain.com/Dataset/ForgiveMe")
print(test)

**Lousy URLs** are URLs that have been created with malicious intent. They are often the precursors to cyberattacks that may happen in the near future. These kinds of urls can hit pretty close to home, leaving each one of us very vulnerable to bad sites that we might visit on purpose or by accident.


**Drive-by download URLS** are URLS that promote the unintended download of software from websites. They could be downloaded when a naive user first clicks on a URL, without knowing the consequences of this action.

**Command and Conquer URLs** are URLs that are linked to malware that connects the target computer to command and control servers. These are different from URLs that can be categorized as malicious as it is not always a virus that concerns to command and control URLs via external or remote servers.

**Phishing URLs** are a form of attack that steals sensitive data, such as **personally identifiable information (PII)**, by either luring the user or disguising the URL as a legitimate or trustworthy URL.

In [ ]:
from pathlib import Path

dataset = Path(Path.cwd()).resolve().parents[1] / "introduction" / "datasets" / "dataset_phishing.csv"

In [ ]:
raw_df = pd.read_csv(dataset)
raw_df.head()

In [ ]:
df = raw_df[['url', 'status']]

In [ ]:
df.loc[df.status == 'phishing'] = 1
df.loc[df.status == 'legitimate'] = 0
df.head()

In [21]:
def get_ipaddress(tokenized_words):
    count = 0
    for element in tokenized_words:
        if str(element).isnumeric():  # fixed from encode() to str()
            count += 1
        else:
            if count >= 4:
                return 1
            count = 0
    return 1 if count >= 4 else 0


### Tokenize URL

In [22]:

def url_tokenize(url):
    """
    Tokenizes a given URL using non-alphanumeric characters and returns:
    1. The average token length (excluding empty tokens),
    2. The number of valid (non-empty) tokens,
    3. The length of the largest token.

    Parameters:
        url (str): The input URL string.

    Returns:
        list: [average_token_length (float), num_tokens (int), max_token_length (int)]
    """
    tokenized_words = re.split(r'\W+', url)
    num_tokens = 0
    total_length = 0
    max_length = 0

    for token in tokenized_words:
        length = len(token)
        if length > 0:
            num_tokens += 1
            total_length += length
            if length > max_length:
                max_length = length

    try:
        avg_length = total_length / num_tokens if num_tokens > 0 else 0
        return [avg_length, num_tokens, max_length]
    except Exception as e:
        return [0, num_tokens, max_length]

    
